In [8]:
import z3
import pprint
import plotly

username = 'eiselesr'
api_key = 'Qf97n8ORXBbEpZEvD6HA'
plotly.tools.set_credentials_file(username=username, api_key=api_key)

a21, a31, a22, a32 = z3.Real('a21'), z3.Real('a31'), z3.Real('a22'), z3.Real('a32')
a23, a33, a24, a34 = z3.Real('a23'), z3.Real('a33'), z3.Real('a24'), z3.Real('a34')
a25, a35 = z3.Real('a25'), z3.Real('a35')

def small_obj(a21):
    uobj = 1/20*a21
    return uobj

def util_obj(a21, a31, a22, a32, a23, a33, a24, a34, a25, a35):
    uobj = (1/20*a21 + 1/20*a31 +
            1/20*a22 + 1/20*a32 +
            1/20*a23 + 1/20*a33 +
            1/20*a24 + 1/20*a34 +
            1/20*a25 + 1/20*a35)
    return uobj

def balance_obj(a21, a31, a22, a32, a23, a33, a24, a34, a25, a35):
    bobj = (
    (z3.simplify((0 + a21*1024 + a31*1024)/4096 - (0 + a21*1024 + a31*1024 + 0 + a22*1024 + a32*1024 + 0 + a23*1024 + a33*1024 +
                                       0 + a24*1024 + a34*1024 + 0 + a25*1024 + a35*1024)/ 20480))**2 +
    (z3.simplify((0 + a22*1024.0 + a32*1024)/4096 - (0 + a21*1024 + a31*1024 + 0 + a22*1024 + a32*1024 + 0 + a23*1024 + a33*1024 +
                                       0 + a24*1024 + a34*1024 + 0 + a25*1024 + a35*1024)/ 20480))**2 +
    (z3.simplify((0 + a23*1024.0 + a33*1024)/4096 - (0 + a21*1024 + a31*1024 + 0 + a22*1024 + a32*1024 + 0 + a23*1024 + a33*1024 +
                                       0 + a24*1024 + a34*1024 + 0 + a25*1024 + a35*1024)/ 20480))**2 + 
    (z3.simplify((0 + a24*1024.0 + a34*1024)/4096 - (0 + a21*1024 + a31*1024 + 0 + a22*1024 + a32*1024 + 0 + a23*1024 + a33*1024 +
                                       0 + a24*1024 + a34*1024 + 0 + a25*1024 + a35*1024)/ 20480))**2 + 
    (z3.simplify((0 + a25*1024.0 + a35*1024)/4096 - (0 + a21*1024 + a31*1024 + 0 + a22*1024 + a32*1024 + 0 + a23*1024 + a33*1024 +
                                       0 + a24*1024 + a34*1024 + 0 + a25*1024 + a35*1024)/ 20480))**2
    )
    return(bobj)

constraints = z3.And([z3.Or(a21 == 0, a21 == 1),
               z3.Or(a22 == 0, a22 == 1),
               z3.Or(a23 == 0, a23 == 1),
               z3.Or(a24 == 0, a24 == 1),
               z3.Or(a25 == 0, a25 == 1),
               z3.Or(a31 == 0, a31 == 1),
               z3.Or(a32 == 0, a32 == 1),
               z3.Or(a33 == 0, a33 == 1),
               z3.Or(a34 == 0, a34 == 1),
               z3.Or(a35 == 0, a35 == 1),
               4096 - (0 + 1024*a21 + 1024*a31) >= 0,
               4096 - (0 + 1024*a22 + 1024*a32) >= 0,
               4096 - (0 + 1024*a23 + 1024*a33) >= 0,
               4096 - (0 + 1024*a24 + 1024*a34) >= 0,
               4096 - (0 + 1024*a25 + 1024*a35) >= 0,
               8192 - (0 + 1024*a21 + 1024*a31) >= 0,
               8192 - (0 + 1024*a22 + 1024*a32) >= 0,
               16384 - (0 + 1024*a23 + 1024*a33) >= 0,
               8192 - (0 + 1024*a24 + 1024*a34) >= 0,
               8192 - (0 + 1024*a25 + 1024*a35) >= 0,
               a21 + a22 + a23 + a24 + a25 > 0,
               a31 + a32 + a33 + a34 + a35 > 0])

opt = z3.Optimize()
opt.assert_exprs(constraints)

def f(q, a21, a31, a22, a32, a23, a33, a24, a34, a25, a35):
    bal_cost = z3.simplify(q*balance_obj(a21, a31, a22, a32, a23, a33, a24, a34, a25, a35))
    util_cost = z3.simplify((1-q)*util_obj(a21, a31, a22, a32, a23, a33, a24, a34, a25, a35))
    return (bal_cost + util_cost)

q = .9
res = opt.minimize(f(q, a21, a31, a22, a32, a23, a33, a24, a34, a25, a35))

opt.check()

print("res.upper: %s" %res.upper())
print("res.lower: %s" %res.lower())

opt_model = opt.model()
display(opt_model)

def checkCost(q, a21, a31, a22, a32, a23, a33, a24, a34, a25, a35):
    
    my_util = z3.simplify(util_obj(a21=a21, a31=a31, 
                                  a22=a22, a32=a32,
                                  a23=a23, a33=a33,
                                  a24=a24, a34=a34,
                                  a25=a25, a35=a35))
    #print("util \n %s " %my_util)
    
    my_balance = z3.simplify(balance_obj(a21=a21, a31=a31, 
                                        a22=a22, a32=a32,
                                        a23=a23, a33=a33,
                                        a24=a24, a34=a34,
                                        a25=a25, a35=a35))
    
    #print("balance \n %s " %my_balance)
    
    f_ = f(q,a21=a21, a31=a31, 
             a22=a22, a32=a32,
             a23=a23, a33=a33,
             a24=a24, a34=a34,
             a25=a25, a35=a35)
    
    
    #print("f \n %s" %f_)
    #print("simplify f : %s" %z3.simplify(f_))
    return f_
    

c1 = (checkCost(q, a21=opt_model[a21], a31=opt_model[a31], 
             a22=opt_model[a22], a32=opt_model[a32], 
             a23=opt_model[a23], a33=opt_model[a33], 
             a24=opt_model[a24], a34=opt_model[a34], 
             a25=opt_model[a25], a35=opt_model[a35]))
print("BalanceCost + UtilCost = %s = %s" %(c1, z3.simplify(c1).as_decimal(5)))

checkCost(q,a21=0, a31=opt_model[a31], 
            a22=opt_model[a22], a32=0, 
            a23=0, a33=opt_model[a33], 
            a24=opt_model[a24], a34=0, 
            a25=0, a35=opt_model[a35])

z30 = z3.RealSort().cast(0)
z31 = z3.RealSort().cast(1)


c2 = (checkCost(q,a21=z30, a31=z31, 
            a22=z31, a32=z30, 
            a23=z30, a33=z31, 
            a24=z31, a34=z30, 
            a25=z30, a35=z31))

print("BalanceCost + UtilCost = %s = %s " %(c2, z3.simplify(c2).as_decimal(5)))

print(a25)

res.upper: oo
res.lower: -1*oo


[a35 = 1,
 a24 = 1,
 a34 = 1,
 a25 = 1,
 a31 = 1,
 a32 = 1,
 a21 = 1,
 a22 = 1,
 a33 = 1,
 a23 = 1]

BalanceCost + UtilCost = 0 + 4999999999999999/100000000000000000 = 0.04999?
BalanceCost + UtilCost = 0 + 4999999999999999/200000000000000000 = 0.02499? 
a25


In [4]:

import operator


def bruteForce(numVars):
    my_list = list()
    my_marking = list()
    for i in range(2**numVars):
        
        marking = format(i, 'b').zfill(numVars) # assign values to variables
        #my_a21 = int(marking[0])
        #print(small_obj(a21=my_a21))
        if marking == format(0,'b').zfill(numVars):
            continue
        my_cost = checkCost(q,a21=z3.RealSort().cast(marking[0]), a31=z3.RealSort().cast(marking[1]), 
                           a22=z3.RealSort().cast(marking[2]), a32=z3.RealSort().cast(marking[3]), 
                           a23=z3.RealSort().cast(marking[4]), a33=z3.RealSort().cast(marking[5]), 
                           a24=z3.RealSort().cast(marking[6]), a34=z3.RealSort().cast(marking[7]), 
                           a25=z3.RealSort().cast(marking[8]), a35=z3.RealSort().cast(marking[9]))
        my_cost_py = z3.simplify(my_cost).as_decimal(5)
        my_list.append(my_cost_py)
        my_marking.append(marking)
        
        #if (i < 20) or (i>1000):
        #    print(my_cost_py, marking)
        #if (i>20):
         #   break
        #print(my_cost_py)
    #print(max(my_list), min(my_list))
    max_index, max_value = max(enumerate(my_list), key=operator.itemgetter(1))
    min_index, min_value = min(enumerate(my_list), key=operator.itemgetter(1))
    #return(my_marking[max_index], max_value, my_marking[min_index], min_value)
    
    marking = my_marking[min_index]
    my_cost = checkCost(q,a21=z3.RealSort().cast(marking[0]), a31=z3.RealSort().cast(marking[1]), 
                           a22=z3.RealSort().cast(marking[2]), a32=z3.RealSort().cast(marking[3]), 
                           a23=z3.RealSort().cast(marking[4]), a33=z3.RealSort().cast(marking[5]), 
                           a24=z3.RealSort().cast(marking[6]), a34=z3.RealSort().cast(marking[7]), 
                           a25=z3.RealSort().cast(marking[8]), a35=z3.RealSort().cast(marking[9]))
    
    return(my_marking[min_index], my_cost, z3.simplify(my_cost).as_decimal(5))

    
    


marking, cost, costsum  = bruteForce(10)
#print("q= %s, res= %s" %(q, res))
print("q= %s, marking = %s, balanceCost + UtilCost=%s, sum = %s" %(q, marking, cost, costsum))
#print("BalanceCost + UtilCost = %s" %c1)

q= 0.1, marking = 0000000001, balanceCost + UtilCost=1/200 + 9/200, sum = 0.05


In [5]:
for x in range(0, 11):
    q = x/10
    marking, cost, costsum  = bruteForce(10)
    print("q= %s, marking = %s, balanceCost + UtilCost=%s, sum = %s" %(q, marking, cost, costsum))

q= 0.0, marking = 0000000001, balanceCost + UtilCost=0 + 1/20, sum = 0.05
q= 0.1, marking = 0000000001, balanceCost + UtilCost=1/200 + 9/200, sum = 0.05
q= 0.2, marking = 0000000001, balanceCost + UtilCost=1/100 + 1/25, sum = 0.05
q= 0.3, marking = 0000000001, balanceCost + UtilCost=3/200 + 7/200, sum = 0.05
q= 0.4, marking = 0000000001, balanceCost + UtilCost=1/50 + 3/100, sum = 0.05
q= 0.5, marking = 0000000001, balanceCost + UtilCost=1/40 + 1/40, sum = 0.05
q= 0.6, marking = 0000000001, balanceCost + UtilCost=3/100 + 1/50, sum = 0.05
q= 0.7, marking = 0000000001, balanceCost + UtilCost=7/200 + 7500000000000001/500000000000000000, sum = 0.05000?
q= 0.8, marking = 0000000001, balanceCost + UtilCost=1/25 + 4999999999999999/500000000000000000, sum = 0.04999?
q= 0.9, marking = 0101010101, balanceCost + UtilCost=0 + 4999999999999999/200000000000000000, sum = 0.02499?
q= 1.0, marking = 0101010101, balanceCost + UtilCost=0 + 0, sum = 0


In [12]:
constraints

And(Or(a21 == 0, a21 == 1),
    Or(a22 == 0, a22 == 1),
    Or(a23 == 0, a23 == 1),
    Or(a24 == 0, a24 == 1),
    Or(a25 == 0, a25 == 1),
    Or(a31 == 0, a31 == 1),
    Or(a32 == 0, a32 == 1),
    Or(a33 == 0, a33 == 1),
    Or(a34 == 0, a34 == 1),
    Or(a35 == 0, a35 == 1),
    4096 - (0 + 1024*a21 + 1024*a31) >= 0,
    4096 - (0 + 1024*a22 + 1024*a32) >= 0,
    4096 - (0 + 1024*a23 + 1024*a33) >= 0,
    4096 - (0 + 1024*a24 + 1024*a34) >= 0,
    4096 - (0 + 1024*a25 + 1024*a35) >= 0,
    8192 - (0 + 1024*a21 + 1024*a31) >= 0,
    8192 - (0 + 1024*a22 + 1024*a32) >= 0,
    16384 - (0 + 1024*a23 + 1024*a33) >= 0,
    8192 - (0 + 1024*a24 + 1024*a34) >= 0,
    8192 - (0 + 1024*a25 + 1024*a35) >= 0,
    a21 + a22 + a23 + a24 + a25 > 0,
    a31 + a32 + a33 + a34 + a35 > 0)

In [100]:
#a21, a22, a23, a24, a25 = z30, z30, z30, z30, z30
#a31, a32, a33, a34, a35 = z31, z31, z31, z31, z31

a21, a31, a22, a32 = z3.Real('a21'), z3.Real('a31'), z3.Real('a22'), z3.Real('a32')
a23, a33, a24, a34 = z3.Real('a23'), z3.Real('a33'), z3.Real('a24'), z3.Real('a34')
a25, a35 = z3.Real('a25'), z3.Real('a35')

constraints = z3.And([
    z3.Or(a21 == 0, a21 == 1),
    z3.Or(a22 == 1, a22 == 1),
    z3.Or(a23 == 0, a23 == 0),
    z3.Or(a24 == 1, a24 == 1),
    z3.Or(a25 == 0, a25 == 0),
    z3.Or(a31 == 1, a31 == 1),
    z3.Or(a32 == 0, a32 == 0),
    z3.Or(a33 == 1, a33 == 1),
    z3.Or(a34 == 0, a34 == 0),
    z3.Or(a35 == 1, a35 == 1),
    4096 - (0 + 1024*a21 + 1024*a31) >= 0,
    4096 - (0 + 1024*a22 + 1024*a32) >= 0,
    4096 - (0 + 1024*a23 + 1024*a33) >= 0,
    4096 - (0 + 1024*a24 + 1024*a34) >= 0,
    4096 - (0 + 1024*a25 + 1024*a35) >= 0,
    8192 - (0 + 1024*a21 + 1024*a31) >= 0,
    8192 - (0 + 1024*a22 + 1024*a32) >= 0,
    16384 - (0 + 1024*a23 + 1024*a33) >= 0,
    8192 - (0 + 1024*a24 + 1024*a34) >= 0,
    8192 - (0 + 1024*a25 + 1024*a35) >= 0,
    a21 + a22 + a23 + a24 + a25 > 0,
    a31 + a32 + a33 + a34 + a35 > 0
])



In [101]:
s = z3.Solver()

In [102]:
#opt = z3.Optimize()
#opt.assert_exprs(constraints)
#opt.check()
s = z3.Solver()
s.add(constraints)
s.check()
#s.unsat_core()

sat

In [103]:
m = s.model()
display(m)
print ("a21 = %s" % m[a21])

[a23 = 0,
 a32 = 0,
 a21 = 1,
 a22 = 1,
 a33 = 1,
 a35 = 1,
 a24 = 1,
 a34 = 0,
 a25 = 0,
 a31 = 1]

a21 = 1


In [104]:
c1 = (checkCost(q, a21=m[a21], a31=m[a31], 
             a22=m[a22], a32=m[a32], 
             a23=m[a23], a33=m[a33], 
             a24=m[a24], a34=m[a34], 
             a25=m[a25], a35=m[a35]))
print("BalanceCost + UtilCost = %s = %s" %(c1, z3.simplify(c1).as_decimal(5)))

BalanceCost + UtilCost = 9/200 + 14999999999999997/500000000000000000 = 0.07499?


In [105]:
a21, a31, a22, a32 = z3.Real('a21'), z3.Real('a31'), z3.Real('a22'), z3.Real('a32')
a23, a33, a24, a34 = z3.Real('a23'), z3.Real('a33'), z3.Real('a24'), z3.Real('a34')
a25, a35 = z3.Real('a25'), z3.Real('a35')

opt = z3.Optimize()
opt.assert_exprs(constraints)

In [106]:
q = .9
res = opt.minimize(f(q, a21, a31, a22, a32, a23, a33, a24, a34, a25, a35))

opt.check()

print("res.upper: %s" %res.upper())
print("res.lower: %s" %res.lower())

opt_model = opt.model()
display(opt_model)

res.upper: oo
res.lower: -1*oo


[a21 = 1,
 a35 = 1,
 a34 = 0,
 a33 = 1,
 a32 = 0,
 a31 = 1,
 a25 = 0,
 a24 = 1,
 a23 = 0,
 a22 = 1]

In [3]:
import z3
a21 = z3.Real('a21')
a22 = z3.Real('a22')
constraints = z3.And([
    z3.Or(a21 == 0, a21 == 1),
    z3.Or(a22 == 0, a22 == 1)])

In [46]:
opt = z3.Optimize()
opt.assert_exprs(constraints)
opt.add((a22 + a21)>0)
opt.add((a22 + a21)<10)
mx = opt.maximize(a22 + a21)


In [47]:
opt.check()

sat

In [48]:
print("mx.upper: %s" %mx.upper())
print("mx.lower: %s" %mx.lower())



mx.upper: 2
mx.lower: 2


In [49]:
opt_model = opt.model()
display(opt_model)

[a21 = 1, a22 = 1]

In [59]:
a21 = z3.Real('a21')
a22 = z3.Real('a22')
constraints = z3.And([
    z3.Or(a21 == 0, a21 == 1),
    z3.Or(a22 == 0, a22 == 1)])

In [60]:
opt = z3.Optimize()
opt.assert_exprs(constraints)
opt.add((a22**2 + a21**2)>0)
opt.add((a22**2 + a21**2)<10)
mn = opt.minimize(a22**2 + a21**2)

In [64]:
print(opt.check())
print(opt.reason_unknown())
print("mn.upper: %s" %mn.upper())
print("mn.lower: %s" %mn.lower())

unknown
(incomplete (theory arithmetic))
mn.upper: oo
mn.lower: -1*oo


In [62]:
opt_model = opt.model()
display(opt_model)

[a21 = 1, a22 = 1]

In [ ]:
# why it won't work
# https://stackoverflow.com/questions/38386570/z3opt-python-minimizing-square
# https://stackoverflow.com/search?q=z3+incomplete+%28theory+arithmetic%29
# https://github.com/Z3Prover/z3/issues?utf8=✓&q=optimize

In [4]:

s = z3.Solver()
s.add(constraints)
s.add((a22**2 + a21**2)>0)
s.add((a22**2 + a21**2)<10)
s.check()


sat

In [5]:
m = s.model()
display(m)

[a21 = 0, a22 = 1]